# import 

In [3]:
import cv2

In [4]:
import random

In [5]:
import os

In [6]:
from fastprogress.fastprogress import progress_bar

In [7]:
import numpy as np

In [8]:
from scipy.ndimage.interpolation import map_coordinates

In [9]:
from scipy.ndimage.filters import gaussian_filter

# functions

In [10]:
# 关注的水体的灰度值。
#0 未标注。 1 空地。 2 建筑 。3 水体。4 道路
TYP_WATER = 3

In [11]:
#水体占比在这个值一下的认为是空的
BLANK_TH = 0.04

In [49]:
#https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
# Function to distort image
def elastic_transform(image, image_mask, alpha, sigma, alpha_affine, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    #import pdb; pdb.set_trace()
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    #print('dbg0', image.shape, random_state)
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    #print('dbg3', image_mask.shape, (image_mask[None]).shape)
    if image_mask is not None:
        #print('dbg5', image_mask.shape)
        #image_mask = cv2.cvtColor(image_mask, cv2.COLOR_GRAY2RGB)
        image_mask = cv2.warpAffine(image_mask, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
        
    #print('dbg1', random_state.rand(*shape) * 2 - 1, sigma, alpha)
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)
    #print('dbg7')
    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape) \
            , map_coordinates(image_mask, indices, order=1, mode='reflect').reshape(shape) if image_mask is not None else None
            

In [50]:
def gen_dataset(imgfns, maskfns, output_dir = 'label/dataset_20200708', imgdir = 'image'
                , maskdir = 'mask', img_sz = (512, 512), ds_sz = 1000, blank_pct = 0.2
                , suffix = '.png', data_aug = [], dbg = []):
    '''
    生成数据集
    参数：
        imgfns：大图片文件名列表
        maskfns：对应的mask文件名列表。与上一个参数里面的文件一一对应
        output_dir：输出数据集目录
        imgdir：输出数据集中存放图片的子目录。
        maskdir：输出数据集中存放mask图片的子目录
        img_sz：提取的小图的大小,内容为(width, height)
        ds_sz：生成图片数量
        blank_pct：无水体图片数量比例
        suffix：生成图片文件扩展名
        data_aug:数据增强
            参数：
                'elastic_tfm':elastic transfrom。在截取到一个图像后应用到图像和mask上，生成新的图像保存。
                                                 也就是会多生成额外的一份数据，会造成数据集增大一倍。
                                                 运行缓慢，生成200张图片加上变换需要4分8秒，不叫只需要2秒。
        dbg：调试用
    '''
    assert len(imgfns) == len(maskfns)
    #检查创建目录
    for sdir in ['', imgdir, maskdir]:
        if not os.path.exists(os.path.join(output_dir, sdir)):
            os.makedirs(os.path.join(output_dir, sdir))
    total_cnt = 0#产出计数
    imgs = []
    for imgfn, maskfn in zip(imgfns, maskfns):
        img = cv2.imread(imgfn)
        #mask = (cv2.imread(maskfn, cv2.IMREAD_GRAYSCALE) == TYP_WATER).astype(int) * 1
        maskr = cv2.imread(maskfn)
        mask = (cv2.cvtColor(maskr, cv2.COLOR_RGB2GRAY) == TYP_WATER).astype(int) * 1
        #print(imgfn, maskfn, img.shape, mask.shape)
        assert img.shape[ : 2] == mask.shape[ : 2]
        imgs += [(img, mask, maskr)]
    
    #当前大图需要产生的空白数据图片的数量
    DS_SZ_BLANK = int(ds_sz * blank_pct)        
    #需要产生的有效图片的数量
    DS_SZ_VALID = ds_sz - DS_SZ_BLANK

    validcnt = 0 #当前图片产生的数量
    blankcnt = 0#当前产出的空白数据集的数量
    #print('dbg1', DS_SZ_1IMG, DS_SZ_1IMG_BLANK, DS_SZ_1IMG_VALID)
    
    #随机生成一个问题是分类区域占比太少的情况下会导致前期生成的大部分都是空白图片。    #    
    #RNDIDX=True，每次生成的图片名序号是随机数。分布在0-ds_sz之间。
    #RNDIDX=False，每次生成的是不是空白是实现确定的，如果对不上就重新分割。
    #会略微慢一些，可忽略，10000张耗时130s:134s。
    RNDIDX = False    
    gen_cats = [i for i in range(ds_sz)]    
    random.shuffle(gen_cats)
    #print('gen_idxs', gen_cats)
    if RNDIDX:
        gen_idxs = gen_cats#仅仅是名字看起来更合适
    
    pbar = progress_bar(range(ds_sz))
    pbar.comment = '生成中'
    for p in pbar:
        #print('dbg0==')
        while validcnt < DS_SZ_VALID or blankcnt < DS_SZ_BLANK:
            idx = random.randint(0, 2)
            #print(idx, imgfns[idx])
            img, mask, maskr = imgs[idx]
            dsx = random.randint(0, img.shape[0] - img_sz[1])
            dsy = random.randint(0, img.shape[1] - img_sz[0])
            dsimg = img[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]            
            dsmask = mask[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]        
            area = cv2.countNonZero(dsmask)
            
            if not RNDIDX and RNDIDX is not None:
                #本次想要生产的类别.用数值来确定。
                cat = gen_cats[total_cnt]
                #cat = np.random.randint(0, ds_sz)
            #print('dbg1==', area, area / (img_sz[0] * img_sz[1]), cat, idx)
            #根据比例判断一下是否有效的
            if area / (img_sz[0] * img_sz[1]) > BLANK_TH:
                #print(validcnt, total_cnt, ds_sz % len(imgfns))
                #这次准备生成的是空白的，得到的是有内容的
                if not RNDIDX and RNDIDX is not None:
                    if cat < DS_SZ_BLANK:
                        continue
                #超过了数量的放弃.并且保证到达整数
                if validcnt >= DS_SZ_VALID:
                    continue
                validcnt += 1
            else:
                #这次应该是有效内容，但是得到的是空白的
                if not RNDIDX and RNDIDX is not None:
                    if cat >= DS_SZ_BLANK:
                        continue
                if blankcnt >= DS_SZ_BLANK:
                    continue
                blankcnt += 1
            #print('dbg2==')
            dsimg_da, dsmask_da = None, None
            if 'elastic_tfm' in data_aug:
                dsimg_da, dsmask_da = elastic_transform(dsimg, maskr[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]
                                            , dsimg.shape[1] * 2, dsimg.shape[1] * 0.08, dsimg.shape[1] * 0.08)
                
            #print('dbg3==')
            if RNDIDX:
                cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsimg)
                cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsmask)
                if dsimg_da is not None:
                    cv2.imwrite(os.path.join(output_dir, imgdir, '%05d_da.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsimg_da)
                    cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask_da.%s' % (gen_idxs[total_cnt], suffix.split('.')[-1])), dsmask_da)
            else:
                cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (total_cnt, suffix.split('.')[-1])), dsimg)
                cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (total_cnt, suffix.split('.')[-1])), dsmask)
                if dsimg_da is not None:
                    cv2.imwrite(os.path.join(output_dir, imgdir, '%05d_da.%s' % (total_cnt, suffix.split('.')[-1])), dsimg_da)
                    cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask_da.%s' % (total_cnt, suffix.split('.')[-1])), dsmask_da)

            total_cnt += 1
            #print('dbg8-----------------')
            break


# test

In [14]:
imgfns = ['data/src/image/1.png', 'data/src/image/2.png', 'data/src/image/3.png']

In [15]:
maskimgfns = ['data/label/train1_labels_8bits.png', 'data/label/train2_labels_8bits.png'
             , 'data/label/train3_labels_8bits.png']

In [51]:
gen_dataset(imgfns, maskimgfns, ds_sz = 200, suffix='png', output_dir = 'label/dataset_20200713') 

In [52]:
gen_dataset(imgfns, maskimgfns, ds_sz = 200, suffix='png', output_dir = 'label/dataset_20200713', data_aug = ['elastic_tfm']) 

# export